# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""15128386783610400428""","""14904047005973763827""",1.0478e18,"""../../../../data/SampleHDS.jso…","""431429164890461189""","""8350824916825971131""",false
"""442276805140003574""","""2310988936302560128""",1.1737e19,"""../../../../data/SampleHDS.jso…","""431429164890461189""","""2393804384196280001""",true
"""1798927304730299411""","""7460695131425925965""",1.6409e19,"""../../../../data/SampleHDS.jso…","""6927954986707063009""","""8350824916825971131""",false
"""442276805140003574""","""18095773661977127584""",1.3266e19,"""../../../../data/SampleHDS.jso…","""3602085178320060768""","""8350824916825971131""",false
"""15128386783610400428""","""18363049598776861194""",3.3553e18,"""../../../../data/SampleHDS.jso…","""3602085178320060768""","""18105090119304563582""",true
"""1798927304730299411""","""16826739054097183027""",1.2817e19,"""../../../../data/SampleHDS.jso…","""16811186771056360201""",null,false
"""1798927304730299411""","""1245122812748066701""",1.3382e19,"""../../../../data/SampleHDS.jso…","""3602085178320060768""","""18105090119304563582""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""670557626058934808""","""10288292485256187716""",1.3116e19,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""866140233526014703""","""Rejected"""
"""5850558693346106391""","""16741978433881023045""",6.3209e18,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""17089285705642195773""","""Accepted"""
"""2315346365847169022""","""4702590790846564255""",3.3727e18,"""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""866140233526014703""","""Rejected"""
"""5850558693346106391""","""583273369182336875""",1.0667e19,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""866140233526014703""","""Rejected"""
"""670557626058934808""","""9039203270551688166""",1.3752e19,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""8406541023126081662""","""Accepted"""
"""2315346365847169022""","""921061475956714942""",1.7741e19,"""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""2315346365847169022""","""6574963661835677873""",2.1310e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""8406541023126081662""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""5035724385132705446""","""14067819858754802416""",6.3138e18,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""12216236272550620802""","""Rejected"""
"""6897197347988935628""","""3104638465187387018""",7.0165e18,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""9476720508035290013""","""Accepted"""
"""13724484791412223870""","""9425241054616201105""",1.1852e19,"""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""12216236272550620802""","""Rejected"""
"""6897197347988935628""","""13767019686859962108""",8.5816e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""12216236272550620802""","""Rejected"""
"""5035724385132705446""","""17242243801687989858""",1.6261e19,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""9499643244147099817""","""Accepted"""
"""13724484791412223870""","""3081796772729088134""",5.5363e18,"""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""13724484791412223870""","""4149622697983828247""",1.0044e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""9499643244147099817""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_City=PREDICTOR_1
Customer_CLV=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""12594641850879923397""","""4950195631896481532""",1.7733e18,"""FirstMortgage30yr""","""3398926481566838359""","""Rejected"""
"""15279087386420065353""","""1004777359529882778""",1.4916e18,"""FirstMortgage30yr""","""7524119428776917905""","""Accepted"""
"""465129257510119162""","""11705309608503000751""",1.7530e19,"""MoneyMarketSavingsAccount""","""3398926481566838359""","""Rejected"""
"""15279087386420065353""","""12179228032171056613""",4.7311e17,"""BasicChecking""","""3398926481566838359""","""Rejected"""
"""12594641850879923397""","""12978907945199555118""",5.4882e18,"""BasicChecking""","""17998643697575935178""","""Accepted"""
"""465129257510119162""","""14070306217976391586""",1.7787e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""465129257510119162""","""1672023217033473558""",1.8042e19,"""BasicChecking""","""17998643697575935178""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'